In [ ]:
from google.colab import drive
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras import activations
from tensorflow.keras import backend as K
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from sklearn.utils import shuffle
import shutil
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

def organize_datasets(original_path, ratio=0.2):
    target_path_1 = '/content/drive/My Drive/Colab Notebooks/train'
    target_path_2 = '/content/drive/My Drive/Colab Notebooks/validation'
    reader = pd.read_csv('/content/drive/My Drive/Colab Notebooks/train.csv')
    train_df = pd.DataFrame(reader)
    shuffle(train_df)
    n = 160
    val = train_df[:n]
    train = train_df[n:]
    for row in train.iloc[:,0]:
        if os.path.exists(target_path_1) :
            full_path = os.path.join(original_path, row)
            if os.path.exists(full_path):
                shutil.copy(full_path,target_path_1)
        else :
            os.makedirs(target_path_1)
            full_path = os.path.join(original_path, row)
            if os.path.exists(full_path):
                shutil.copy(full_path,target_path_1)
    for row in val.iloc[:,0]:
        if os.path.exists(target_path_2) :
            full_path = os.path.join(original_path, row)
            shutil.copy(full_path,target_path_2)
        else :
            os.makedirs(target_path_2)
            full_path = os.path.join(original_path, row)
            shutil.copy(full_path,target_path_2)

    target_path = '/content/drive/My Drive/Colab Notebooks/test'
    reader = pd.read_csv('/content/drive/My Drive/Colab Notebooks/testName.csv')
    test_df = pd.DataFrame(reader)
    for row in test_df.iloc[:,0]:
        if os.path.exists(target_path) :
            full_path = os.path.join(original_path, row)
            if os.path.exists(full_path):
                shutil.copy(full_path,target_path)
        else :
            os.makedirs(target_path)
            full_path = os.path.join(original_path, row)
            if os.path.exists(full_path):
                shutil.copy(full_path,target_path)          
    return train, val, test_df


In [ ]:
# Load training images from Google drive directories

trdf, vadf, tedf = organize_datasets('/content/drive/My Drive/Colab Notebooks/totalDepthPic')
base_dir = '/content/drive/My Drive/Colab Notebooks'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')
columns=['Total rebars']

datagen = ImageDataGenerator(rescale=1./255,
                             horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_dataframe(trdf, train_dir, x_col='Pic Name', y_col=columns, 
                                              target_size=(150, 150), batch_size=30, class_mode='other')
validation_generator = test_datagen.flow_from_dataframe(vadf, validation_dir, x_col='Pic Name', y_col=columns,
                                                        target_size=(150, 150), batch_size=20, class_mode='other')
test_generator = test_datagen.flow_from_dataframe(tedf, test_dir, x_col='Pic Name',
                                                  batch_size=1, shuffle=False, class_mode=None, target_size=(150, 150))
STEP_SIZE_TRAIN=40
STEP_SIZE_VALID=20
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

In [ ]:
# Pretrained model: InceptionV3 with random initialization

conv_base1 = InceptionV3(include_top=False, input_shape=(150, 150, 3), weights=None,)
model1 = models.Sequential()
model1.compile(optimizer='Adam', loss='mean_squared_error', metrics=[metrics.mae])
model1.add(conv_base1)
model1.add(layers.Flatten())
model1.add(layers.Dropout(0.5))
model1.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model1.add(layers.Dense(1, activation='linear'))

# Train model
history = model1.fit_generator(generator=train_generator, 
                               steps_per_epoch=STEP_SIZE_TRAIN, 
                               validation_data=validation_generator, 
                               validation_steps=STEP_SIZE_VALID, 
                               epochs=30)
model1.save('/content/drive/My Drive/Colab Notebooks/model1.h5')

In [ ]:
# Pretrained model: InceptionV3 with pre-training on ImageNet

datagen = ImageDataGenerator(rescale=1./255)

conv_base2 = Xception(include_top=False, input_shape=(150, 150, 3))
model2 = models.Sequential()
model2.compile(optimizer='Adam', oss='mean_squared_error', metrics=[metrics.mae])
model2.add(conv_base2)
model2.add(layers.Flatten())
model2.add(layers.Dropout(0.5))
model2.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model2.add(layers.Dense(1, activation='linear'))

model2.fit_generator(generator=train_generator, 
                     steps_per_epoch=STEP_SIZE_TRAIN, 
                     validation_data=validation_generator, 
                     validation_steps=STEP_SIZE_VALID, 
                     epochs=30)
model2.save('/content/drive/My Drive/Colab Notebooks/model2.h5')

In [ ]:
# Pretrained model: VGG19

datagen = ImageDataGenerator(rescale=1./255, vertical_flip=True, horizontal_flip=True)

conv_base3 = VGG19(include_top=False, input_shape=(150, 150, 3))
model3 = models.Sequential()
model3.compile(optimizer='Adam', 
              loss='mean_squared_error', 
              metrics=[metrics.mae])
model3.add(conv_base3)
model3.add(layers.Flatten())
model3.add(layers.Dropout(0.5))
model3.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model3.add(layers.Dense(1, activation='linear'))

model3.fit_generator(generator=train_generator, 
                     steps_per_epoch=STEP_SIZE_TRAIN, 
                     validation_data=validation_generator, 
                     validation_steps=STEP_SIZE_VALID, 
                     epochs=30)
model3.save('/content/drive/My Drive/Colab Notebooks/model3.h5')

In [ ]:
# Pretrained model: ResNet50V2

datagen = ImageDataGenerator(rescale=1./255, vertical_flip=True)

conv_base4 = ResNet50V2(include_top=False, input_shape=(150, 150, 3))
model4 = models.Sequential()
model4.compile(optimizer='Adam', 
              loss='mean_squared_error', 
              metrics=[metrics.mae])
model4.add(conv_base4)
model4.add(layers.Flatten())
model4.add(layers.Dropout(0.5))
model4.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model4.add(layers.Dense(1, activation='linear'))

model4.fit_generator(generator=train_generator, 
                     steps_per_epoch=STEP_SIZE_TRAIN, 
                     validation_data=validation_generator, 
                     validation_steps=STEP_SIZE_VALID, 
                     epochs=30)
model4.save('/content/drive/My Drive/Colab Notebooks/model4.h5')

In [ ]:
# Pretrained model: InceptionResNetV2

datagen = ImageDataGenerator(rescale=1./255)

conv_base5 = InceptionResNetV2(include_top=False, input_shape=(150, 150, 3))
model5 = models.Sequential()
model5.compile(optimizer='Adam', 
              loss='mean_squared_error', 
              metrics=[metrics.mae])
model5.add(conv_base5)
model5.add(layers.Flatten())
model5.add(layers.Dropout(0.5))
model5.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model5.add(layers.Dense(1, activation='linear'))

model5.fit_generator(generator=train_generator, 
                     steps_per_epoch=STEP_SIZE_TRAIN, 
                     validation_data=validation_generator, 
                     validation_steps=STEP_SIZE_VALID, 
                     epochs=30)
model5.save('/content/drive/My Drive/Colab Notebooks/model5.h5')

In [ ]:
# Pretrained model: NASNetLarge with random initialization

datagen = ImageDataGenerator(rescale=1./255, vertical_flip=True, horizontal_flip=True)

conv_base6 = NASNetLarge(include_top=False, input_shape=(150, 150, 3), weights=None)
model6 = models.Sequential()
model6.compile(optimizer='Adam', 
              loss='mean_squared_error', 
              metrics=[metrics.mae])
model6.add(conv_base6)
model6.add(layers.Flatten())
model6.add(layers.Dropout(0.5))
model6.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model6.add(layers.Dense(1, activation='linear'))

model6.fit_generator(generator=train_generator, 
                     steps_per_epoch=STEP_SIZE_TRAIN, 
                     validation_data=validation_generator, 
                     validation_steps=STEP_SIZE_VALID, 
                     epochs=30)
model6.save('/content/drive/My Drive/Colab Notebooks/model6.h5')

In [ ]:
# Predict

STEP_SIZE_TRAIN=100
STEP_SIZE_VALID=50
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

model1 = models.load_model('/content/drive/My Drive/Colab Notebooks/model1.h5')
test_generator.reset()
pred = model1.predict_generator(test_generator, steps=STEP_SIZE_TEST, verbose=2)
predictions = pred
results1=pd.DataFrame(predictions, columns=columns)
results1["Pic Name"] = test_generator.filenames
ordered_cols = ["Pic Name"] + columns
results1 = results1[ordered_cols]

model2 = models.load_model('/content/drive/My Drive/Colab Notebooks/model2.h5')
test_generator.reset()
pred = model2.predict_generator(test_generator, steps=STEP_SIZE_TEST, verbose=2)
predictions = pred
results2=pd.DataFrame(predictions, columns=columns)
results2["Pic Name"] = test_generator.filenames
ordered_cols = ["Pic Name"] + columns
results2 = results2[ordered_cols]

model3 = models.load_model('/content/drive/My Drive/Colab Notebooks/model3.h5')
test_generator.reset()
pred = model3.predict_generator(test_generator, steps=STEP_SIZE_TEST, verbose=2)
predictions = pred
results3=pd.DataFrame(predictions, columns=columns)
results3["Pic Name"] = test_generator.filenames
ordered_cols = ["Pic Name"] + columns
results3 = results3[ordered_cols]

model4 = models.load_model('/content/drive/My Drive/Colab Notebooks/model4.h5')
test_generator.reset()
pred = model4.predict_generator(test_generator, steps=STEP_SIZE_TEST, verbose=2)
predictions = pred
results4=pd.DataFrame(predictions, columns=columns)
results4["Pic Name"] = test_generator.filenames
ordered_cols = ["Pic Name"] + columns
results4 = results4[ordered_cols]

model5 = models.load_model('/content/drive/My Drive/Colab Notebooks/model5.h5')
test_generator.reset()
pred = model5.predict_generator(test_generator, steps=STEP_SIZE_TEST, verbose=2)
predictions = pred
results5=pd.DataFrame(predictions, columns=columns)
results5["Pic Name"] = test_generator.filenames
ordered_cols = ["Pic Name"] + columns
results5 = results5[ordered_cols]

model6 = models.load_model('/content/drive/My Drive/Colab Notebooks/model6.h5')
test_generator.reset()
pred = model6.predict_generator(test_generator, steps=STEP_SIZE_TEST, verbose=2)
predictions = pred
results6=pd.DataFrame(predictions, columns=columns)
results6["Pic Name"] = test_generator.filenames
ordered_cols = ["Pic Name"] + columns
results6 = results6[ordered_cols]

In [ ]:
# Ensemble

re = []
for i in range(len(results1.iloc[:,0])):
    a = results1.iloc[i,-1] 
    b = results2.iloc[i,-1]
    c = results3.iloc[i,-1]
    d = results4.iloc[i,-1]
    e = results5.iloc[i,-1]
    f = results6.iloc[i,-1]
    number = round(((a+b+c+d+e+f)/6))
    re.append(number)
results=pd.DataFrame(re, columns=columns)
results["Pic Name"] = test_generator.filenames
ordered_cols = ["Pic Name"] + columns
results = results[ordered_cols]
results.to_csv('/content/drive/My Drive/Colab Notebooks/results.csv',index=False)